Это домашка сделана на меньшем объёме данных - 20 000, чтобы мой ноутбук не кашлял кровью.  Я использую второй столбец как корпус, а первый - как запросы. Не будет работать без уже распакованног файла сданными

In [1]:
import csv
import pymorphy2
import string
import re
from math import log
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

morph = pymorphy2.MorphAnalyzer()
vectorizer1 = CountVectorizer(binary = True)
vectorizertf = TfidfVectorizer(use_idf=False)

In [2]:
def get_texts():
    corpus = []
    inquiery = []
    lenghths = []
    scores = []
    normal = []

    with open('quora_question_pairs_rus.csv', encoding = 'utf-8') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            normal.append(row[2])
            text = preproc(row[2])
            corpus.append(text)
            inquiery.append(row[1])
            lenghths.append(len(text.split(' ')))
            scores.append(row[3])
            if len(scores) == 20000:
                return [corpus, inquiery, lenghths, scores, normal]

def preproc(data):                      # препроцессинг (убираем знаки препинания и числа, приводим всё к начальной форме)
            data = data.split()        
            text = ''                   # числа есть только в тех информации, в самом скрипте все числа пишутся буквами
            for word in data:
                word = word.strip('[!,.?"]')
                p = morph.parse(word.strip())[0]
                p = p.normal_form
                if p != '-':
                    if re.search(r'\d', p) == None:
                        text  = text + ' ' + p
            return text[1:]    

In [3]:
data = get_texts()

In [4]:
corpus_texts, inquiery_texts, lenghths, scores, normal_texts =  data[0][1:], data[1][1:], data[2][1:], data[3][1:], data[4][1:]
del data

In [5]:
print(corpus_texts[1])
print(inquiery_texts[1])
print(scores[1])

как повысить скорость интернет путём взлом через dns
как я могу увеличить скорость моего интернет-соединения, используя vpn
0


In [6]:
avgdl = np.mean(lenghths) 
k = 2
b = 0.75

In [7]:
corpus_matrix1 = vectorizer1.fit_transform(corpus_texts)
voc = vectorizer1.get_feature_names()
N = len(corpus_matrix1.getnnz(axis = 1))
n = corpus_matrix1.sum(axis=0)
n = n.tolist()[0]

In [8]:
idf = []
for i in range(len(voc)):
    idf_p = log((N-n[i]+0.5) / (n[i]+0.5))
    if idf_p < 0:
            idf_p = 0
    idf.append(idf_p)

In [9]:
del corpus_matrix1

In [10]:
tf = vectorizertf.fit_transform(corpus_texts)

In [11]:
tf = tf.toarray()

In [12]:
additional = []
for i  in lenghths:
    res = k * (1-b+b*(i/avgdl))
    additional.append(res)

In [13]:
k1 = k + 1
for i in range(N):
    for j in range(len(voc)):
        tf[i][j] = (tf[i][j]*k1)/(tf[i][j] + additional[i]) * idf[j]

In [14]:
del additional

теперь, перейдём к "быстрому"способу - создадим массив, из которого будет удобно всё вытаскивать

In [15]:
def bm25_f(inquiery, doc, voc):
    res = 0
    for word in inquiery.split(' '):
        try:
            index = voc.index(word)
        except ValueError:
            res += 0
        else:
            res += tf[doc][index]
    return res

А вот и сам поисковик (выдающий **пять** лучших результатов, через **цикл**)

In [29]:
def search5(inquiery, N, voc):
    d = {}
    inq = preproc(inquiery)
    for i in range(N):
        d[i] = bm25_f (inq, i, voc)
    res = sorted(d, key=d.get, reverse=True)[:5]
    for i in res:
        print(normal_texts[i])
    return

In [30]:
search5('правительство индии', N, voc )

как работает правительство Индии
почему правительство Индии запрещает все
какая разница между парламентом и правительством Индии
каковы характеристики тоталитарного правительства
не принял одобрение правительства


#  Задание 1

не выполнено, так как у меня только один вариант

# Задание 2

In [27]:
def search10(inquiery, N, voc):
    d = {}
    inq = preproc(inquiery)
    for i in range(N):
        d[i] = bm25_f (inq, i, voc)
    res = sorted(d, key=d.get, reverse=True)[:10]
    for i in res:
        print(normal_texts[i], '    результат:', d[i])

In [28]:
search10('рождественские каникулы', N, voc )

каков ваш рождественский список     результат: 7.438534445830369
какой лучший рождественский подарок     результат: 7.438534445830369
как я могу провести летние каникулы осмысленно     результат: 5.365484239973764
какой лучший рождественский подарок вы когда-либо давали     результат: 4.355020046598063
я не хочу покупать маме рождественский подарок, что я должен делать     результат: 3.1175849918074796
вы можете найти рождественскую коляду, которую пела женщина в 1995 году 6 в очень грустном голосе     результат: 2.3089523138721044
я делаю бакалавров в области компьютерных наук, я сделаю с моим 4-м семестром за несколько дней. У меня есть 3 месяца летних каникул вперед. Какой лучший способ я могу использовать эти 3 месяца, чтобы я мог приземлиться на хорошую стажировку в будущем на вершине компания с надписью типа microsoft google facebook и т. д.     результат: 0.43368348190130357
что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад     результат: 0.0
как по

Как видно, 10 не наберётся. Вероятно, в связи с тем, что я считаю на 20к

#  Задание 3

Это задание работает невероятно долго

для этого задания разработаем поиск который ничего не принтит, а вместо этого ретёрнит res

In [33]:
def search5_np(inquiery, N, voc):
    d = {}
    inq = preproc(inquiery)
    for i in range(N):
        d[i] = bm25_f (inq, i, voc)
    res = sorted(d, key=d.get, reverse=True)[:5]
    return res

в обеих колонках есть дубликаты, так что....

In [44]:
d ={}
for i in range(1000):
    if int(scores[i]) == 1:
        if inquiery_texts[i] in d.keys():
            d[inquiery_texts[i]].append(normal_texts[i])
        else:
            d[inquiery_texts[i]] = [normal_texts[i]]

In [45]:
d

{'астрология: я - луна-колпачок из козерога и крышка, поднимающая то, что это говорит обо мне': ['Я тройная луна-козерог и восхождение в козероге, что это говорит обо мне'],
 'как я могу быть хорошим геологом?': ['что я должен делать, чтобы быть великим геологом?'],
 'как мне читать и находить комментарии к YouTube': ['как я могу увидеть все мои комментарии к YouTube'],
 'что может сделать физику легкой для изучения': ['как вы можете легко научиться физике'],
 'какой был ваш первый сексуальный опыт, как': ['какой был ваш первый сексуальный опыт'],
 'что означало бы президентство козыря для нынешних иностранных студентов-магистров по визе f1': ['как президентство козыря повлияет на учеников в настоящее время в нас или планирует учиться у нас'],
 'что означает манипуляция': ['что означает манипуляция'],
 'почему так много пользователей quora публикуют вопросы, на которые легко ответить на google': ['почему люди задают вопросы о quora, на которые можно легко ответить google'],
 'почему ра

In [ ]:
# результат с нормальными параметрами
count = 0
for i in range(1000):
    res = search5_np(inquiery_texts[i], N, voc)
    c = 0
    if  inquiery_texts[i] in d.keys():
        for j in res:
            if normal_texts[j] in d[inquiery_texts[i]]:
                c = 1
    count += c
    
print((count/N)*100)

 Сделаем всё для bm15

In [ ]:
k = 2
b = 0
tf = vectorizertf.fit_transform(corpus_texts)
tf = tf.toarray()

In [ ]:
additional = []
for i  in lenghths:
    res = k * (1-b+b*(i/avgdl))
    additional.append(res)

k1 = k + 1
for i in range(N):
    for j in range(len(voc)):
        tf[i][j] = (tf[i][j]*k1)/(tf[i][j] + additional[i]) * idf[j]

In [ ]:
def bm15_f(inquiery, doc, voc):
    res = 0
    for word in inquiery.split(' '):
        try:
            index = voc.index(word)
        except ValueError:
            res += 0
        else:
            res += tf[doc][index]
    return res

In [ ]:
def search5_np(inquiery, N, voc):
    d = {}
    inq = preproc(inquiery)
    for i in range(N):
        d[i] = bm15_f (inq, i, voc)
    res = sorted(d, key=d.get, reverse=True)[:5]
    return res

In [ ]:
count = 0
for i in range(1000):
    res = search5_np(inquiery_texts[i], N, voc)
    c = 0
    if  inquiery_texts[i] in d.keys():
        for j in res:
            if normal_texts[j] in d[inquiery_texts[i]]:
                c = 1
    count += c
    
print((count/N)*100)

а теперь сделаем всё для bm11

In [ ]:
k = 2
b = 1
tf = vectorizertf.fit_transform(corpus_texts)
tf = tf.toarray()

In [ ]:
additional = []
for i  in lenghths:
    res = k * (1-b+b*(i/avgdl))
    additional.append(res)

k1 = k + 1
for i in range(N):
    for j in range(len(voc)):
        tf[i][j] = (tf[i][j]*k1)/(tf[i][j] + additional[i]) * idf[j]

In [ ]:
def bm11_f(inquiery, doc, voc):
    res = 0
    for word in inquiery.split(' '):
        try:
            index = voc.index(word)
        except ValueError:
            res += 0
        else:
            res += tf[doc][index]
    return res

In [ ]:
def search5_np(inquiery, N, voc):
    d = {}
    inq = preproc(inquiery)
    for i in range(N):
        d[i] = bm11_f (inq, i, voc)
    res = sorted(d, key=d.get, reverse=True)[:5]
    return res

In [ ]:
count = 0
for i in range(1000):
    res = search5_np(inquiery_texts[i], N, voc)
    c = 0
    if  inquiery_texts[i] in d.keys():
        for j in res:
            if normal_texts[j] in d[inquiery_texts[i]]:
                c = 1
    count += c
    
print((count/N)*100)

В общем, все результаты сравнимы